In [83]:
import re
import gensim
from gensim import corpora, models, similarities
import nltk
import pandas as pd
import os
import codecs
import gensim.models.word2vec as w2v

In [2]:
#Fungsi untuk membangun model Word2Vec
def getWord2Vec(toFeed, dim=50):
    return gensim.models.Word2Vec(toFeed, min_count=1,  size=dim)

In [15]:
corpus = pd.read_csv("~/datasets/Kaggle/competitions/clinicaltrials/labeledEligibilitySample1000000.csv", 
                     sep='\t',
                     error_bad_lines=False,
                     header=None)

In [16]:
len(corpus)

1000000

In [23]:
corpus.head()

,0,1
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__0,study interventions are Liposomal doxorubicin ...
2,__label__0,study interventions are BI 836909 . multiple m...
3,__label__0,study interventions are Immunoglobulins . recu...
4,__label__0,study interventions are Paclitaxel . stage ova...


In [29]:
corpus.iloc[:,1].head()
corpus_data= pd.DataFrame(corpus.iloc[:,1])

In [43]:
corpus.shape, type(corpus)

((1000000, 2), pandas.core.frame.DataFrame)

In [50]:
corpus.columns = ['label','text']
corpus.head()

,label,text
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__0,study interventions are Liposomal doxorubicin ...
2,__label__0,study interventions are BI 836909 . multiple m...
3,__label__0,study interventions are Immunoglobulins . recu...
4,__label__0,study interventions are Paclitaxel . stage ova...


In [53]:
corpus_blob = corpus.groupby('label')['text'].apply(' '.join).reset_index()
corpus_blob

,label,text
0,__label__0,study interventions are recombinant CD40-ligan...
1,__label__1,study interventions are Prednisone . patients ...


In [75]:
inclusion_criteria_0 = corpus_blob['text'][0]
len(inclusion_criteria_0),inclusion_criteria_0[0:500]

(83466072,
 'study interventions are recombinant CD40-ligand . melanoma skin diagnosis and no active cns metastases by ct scan or mri study interventions are Liposomal doxorubicin . colorectal cancer diagnosis and cardiovascular study interventions are BI 836909 . multiple myeloma diagnosis and indwelling central venous cateder or willingness to undergo intra venous central line placement study interventions are Immunoglobulins . recurrent fallopian tube carcinoma diagnosis and patients are allowed to receiv')

In [115]:
inclusion_criteria = corpus['text']

In [116]:
stoplist = set('for a of the and to in is at an must be with are but not no none has have other from as prior or except none'.split())
texts = [[word for word in inclusion_criteria.split() if word not in stoplist]
         for inclusion_criteria in inclusion_criteria]

In [120]:
texts[0:10]

[['study',
  'interventions',
  'recombinant',
  'CD40-ligand',
  '.',
  'melanoma',
  'skin',
  'diagnosis',
  'active',
  'cns',
  'metastases',
  'by',
  'ct',
  'scan',
  'mri'],
 ['study',
  'interventions',
  'Liposomal',
  'doxorubicin',
  '.',
  'colorectal',
  'cancer',
  'diagnosis',
  'cardiovascular'],
 ['study',
  'interventions',
  'BI',
  '836909',
  '.',
  'multiple',
  'myeloma',
  'diagnosis',
  'indwelling',
  'central',
  'venous',
  'cateder',
  'willingness',
  'undergo',
  'intra',
  'venous',
  'central',
  'line',
  'placement'],
 ['study',
  'interventions',
  'Immunoglobulins',
  '.',
  'recurrent',
  'fallopian',
  'tube',
  'carcinoma',
  'diagnosis',
  'patients',
  'allowed',
  'receive',
  'required',
  'receive',
  'two',
  'additional',
  'cytotoxic',
  'regimens',
  'management',
  'recurrent',
  'persistent',
  'disease',
  'more',
  'than',
  'one',
  'non',
  'platinum',
  'non',
  'taxane',
  'regimen'],
 ['study',
  'interventions',
  'Paclitaxel

In [121]:
inclusion_criteria = texts.values.tolist()
len(inclusion_criteria),inclusion_criteria[0:500]

AttributeError: 'list' object has no attribute 'values'

In [110]:
len(texts), texts[1:10]

(1000000,
 [['study',
   'interventions',
   'Liposomal',
   'doxorubicin',
   '.',
   'colorectal',
   'cancer',
   'diagnosis',
   'cardiovascular'],
  ['study',
   'interventions',
   'BI',
   '836909',
   '.',
   'multiple',
   'myeloma',
   'diagnosis',
   'indwelling',
   'central',
   'venous',
   'cateder',
   'willingness',
   'undergo',
   'intra',
   'venous',
   'central',
   'line',
   'placement'],
  ['study',
   'interventions',
   'Immunoglobulins',
   '.',
   'recurrent',
   'fallopian',
   'tube',
   'carcinoma',
   'diagnosis',
   'patients',
   'allowed',
   'receive',
   'required',
   'receive',
   'two',
   'additional',
   'cytotoxic',
   'regimens',
   'management',
   'recurrent',
   'persistent',
   'disease',
   'more',
   'than',
   'one',
   'non',
   'platinum',
   'non',
   'taxane',
   'regimen'],
  ['study',
   'interventions',
   'Paclitaxel',
   '.',
   'stage',
   'ovarian',
   'cancer',
   'diagnosis',
   'patients',
   'recovered',
   'effects',
 

In [63]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/akshaychougule/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/akshaychougule/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [111]:
type(texts)

list

In [122]:
#f = codecs.open('C:\Python26\text.txt', 'r', 'utf-8-sig')
#text = f.read()
tok_corp = nltk.word_tokenize(str(texts))

In [114]:
len(tok_corp), tok_corp[1:20]

(59485877,
 ['[',
  "'study",
  "'",
  ',',
  "'interventions",
  "'",
  ',',
  "'recombinant",
  "'",
  ',',
  "'CD40-ligand",
  "'",
  ',',
  "'",
  '.',
  "'",
  ',',
  "'melanoma",
  "'"])

In [124]:
ct2vec = w2v.Word2Vec(
    texts,
    workers=3,
    size=100,
    min_count=5,
    window=5,
    sample=1e-3,
)

In [125]:
print(ct2vec['aml'])

[ 1.0115347   1.996176   -3.0547388   0.1757077  -1.2841983   3.2444172
 -4.443465   -0.13637473  2.2183628  -0.03060763  0.58144134  5.198439
  2.697748   -2.0510092   1.5133778  -0.08234689  1.0744412   1.1355784
 -1.6095499  -0.22257918  1.3326505  -2.2306838   1.4950866  -1.4847589
  0.23418763  0.64909697  2.7352138  -1.657302   -3.7804391   0.18397163
 -3.2278097   4.22745     1.585811   -3.6379879   3.4386363  -2.901265
 -0.7544362  -1.4433676   1.1468799   0.47232136  0.95242745 -1.626939
 -1.2439122   1.3504711  -0.66758156  5.9319468  -0.11947972  1.8505406
 -2.6652834  -3.3176978  -3.5497081   3.377744   -2.9510145  -0.42378274
  1.5605372  -0.338233   -1.2642208  -1.8639265  -2.4487686  -0.97635365
 -1.599603    0.15710193  3.4166036   1.8208599   2.9171405   0.02184244
  3.6964858   1.4126037  -0.80628777 -0.35083935  0.24263956  0.537919
 -1.6473005   1.4782016  -1.3247566  -4.3163157   0.03546025  0.13299026
 -0.65219265  4.6102085   1.2400672  -0.78184205 -1.9304621   1

/Users/akshaychougule/anaconda2/envs/ds-book/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [126]:
ct2vec.most_similar('aml')

/Users/akshaychougule/anaconda2/envs/ds-book/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('mds', 0.7537376284599304),
 ('cml', 0.6268577575683594),
 ('myelodysplasia', 0.6225919723510742),
 ('mpn', 0.6106234192848206),
 ('mps', 0.5599762201309204),
 ('cll', 0.5290195941925049),
 ('rcmd', 0.5251673460006714),
 ('blast', 0.524329662322998),
 ('transformation', 0.5091052055358887),
 ('cmml', 0.49349966645240784)]

In [128]:
ct2vec.most_similar('Sorafenib')

/Users/akshaychougule/anaconda2/envs/ds-book/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('Niacinamide', 0.9932081699371338),
 ('Sunitinib', 0.9272384643554688),
 ('Tremelimumab', 0.9055455327033997),
 ('Bevacizumab', 0.9002907276153564),
 ('Everolimus', 0.8966895341873169),
 ('Interferons', 0.8957058787345886),
 ('Interferon-alpha', 0.8900223970413208),
 ('Sirolimus', 0.8899325132369995),
 ('Vatalanib', 0.8746511936187744),
 ('Trebananib', 0.874604344367981)]